In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stat
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import math

In [4]:
name_dict = pickle.load(open("models/aka_name_dbscan_name_map.pkl", "rb"))
type(name_dict), len(name_dict)

(dict, 10450)

In [5]:
cluster_dict = pickle.load(open("models/aka_name_dbscan_cluster_map.pkl", "rb"))
type(cluster_dict), len(cluster_dict)

(dict, 911)

In [6]:
RAW_NAMES, clusters = pickle.load(open("models/aka_name_dbscan_clusters.pkl", "rb"))
type(RAW_NAMES), len(RAW_NAMES), type(clusters), len(clusters),

(numpy.ndarray, 10450, numpy.ndarray, 10450)

In [8]:
df_canvass = pd.read_csv("data/canvass_records.csv.gz")
print("Read {} canvass records.".format(len(df_canvass)))
df_complaint = pd.read_csv("data/complaint_records.csv.gz")
print("Read {} complaint records.".format(len(df_complaint)))
df_all = pd.concat([df_canvass, df_complaint])
df_all["aka_name"] = df_all["aka_name"].fillna("NULL_NAME")
df_all.head()

Read 50462 canvass records.
Read 17088 complaint records.


,inspection_id,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_id,...,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
0,1981667,1725 W 47TH ST,BELLAS FUENTEZ,CHICAGO,BELLAS FUENTEZ,Restaurant,2017-01-19T00:00:00.000,Canvass,41.808460,1356217,...,0,0,0,0,0,0,0,0,0,0
1,1453500,65 W JACKSON BLVD,CARVERY,CHICAGO,CARVERY,Restaurant,2014-08-26T00:00:00.000,Canvass,41.878046,43330,...,0,0,0,0,0,0,0,0,0,0
2,1102252,11601 W TOUHY AVE,THE GREAT AMERICAN BAGEL/T3 K2,CHICAGO,THE GREAT AMERICAN BAGEL,Restaurant,2012-05-07T00:00:00.000,Canvass,42.008536,1879167,...,0,0,0,0,0,0,0,0,0,0
3,68091,2804 N CLARK ST,Wells Street Popcorn,CHICAGO,Wells Street Popcorn,Restaurant,2010-02-01T00:00:00.000,Canvass,41.932921,1954774,...,0,0,0,0,0,0,0,0,0,0
4,1482983,1847 W ROSCOE ST,FOUR MOON TAVERN,CHICAGO,FOUR MOON TAVERN,Restaurant,2015-09-14T00:00:00.000,Canvass,41.943141,51644,...,0,0,0,0,0,0,0,0,0,0


In [10]:
name_count = df_all.groupby("aka_name")["inspection_id"].count()
cdf = pd.DataFrame(cluster_dict.items(), columns=["Cluster", "Restaurants"])
cdf["Names"] = cdf["Restaurants"].apply(lambda a: len(a))
cdf["Inspections"] = cdf["Restaurants"].apply(lambda a: np.sum([name_count[n] for n in a]))
cdf.sort_values(by="Inspections", ascending=False).head(30)

,Cluster,Restaurants,Names,Inspections
0,0,"[#1 CHINA EXPRESS , LTD, #1 CHOP SUEY, 13 Pins...",3837,24705
1,-1,"[#1 WOK N ROLL, 1 2 3 EXPRESS, 1-94 RIBS & GRI...",3244,16804
804,803,"[SUBWAY, SUBWAY # 23333, SUBWAY #23983, SUBWAY...",22,2632
235,234,"[DUNKIN DONUTS / BASKIN ROBBINS, DUNKIN DONUT...",33,1551
568,567,"[MC DONALD' S # 6888, MC DONALD' S # 22100, M...",54,1449
360,359,"[GT's Fast Food, JIM CHING RESTAURANT, JIM'S O...",43,556
166,165,"[CHIPOTLE, CHIPOTLE MEXICAN GRILL, CHIPOTLE ME...",4,360
376,375,"[HAROLD'S CHICKEN, HAROLD'S CHICKEN SHACK, H...",24,295
417,416,"[INDIE BURGER, THE WING STOP EXPERTS, WINDY CI...",34,221
885,884,"[WENDY'S, WENDY'S #000860, WENDY'S #001110, WE...",8,214


In [16]:
for r in cdf["Restaurants"].values[2:10]:
    print(r)
    print()

['(K)  NEW  RESTAURANT', 'K-KITCHEN']

['11 DINING', '11 DINING, LLC']

['1ST DOWN FRANK', '1ST DOWN FRANKS', 'IST DOWN FRANKS']

['2015 SEE THRU CHINESE KITCHEN', 'SEE THRU CHINESE', 'SEE THRU CHINESE KITCHEN', 'SEE THRU CHINESE KITCHEN #13', 'SEE THRU CHINESE KITCHEN #15', 'SEE THRU CHINESE KITCHEN #21', 'SEE THRU CHINESE KITCHEN INC', 'SEE THRU CHINESE RESTAURANT', 'See Thru Chinese Kitchen']

['25 DEGREES', '6 DEGREES']

['33 CLUB', 'B&H CLUB', 'CLUB 162', 'CLUB 40', 'CLUB DE NUTRICION', 'CLUB DESSERTS S-212', 'CLUB EXPRESS 229', 'CLUB EXPRESS S-206', 'CLUB EXPRESS S-223', 'CLUB EXPRESS S-229', 'CLUB KITCHEN', 'CLUB LAGO', 'CLUB LUCKY', "D'CUBA RESTAURANT", 'THE CUB HOUSE #1006', 'THE CUB HOUSE #1009', 'THE CUB HOUSE #2018', 'The Cub House #3019']

['4 BANDERAS', 'BANADIR RESTAURANT', 'BANDERA']

['4 HERMANOS', '4 HERMANOS TOO', 'HADERMOAT RESTAURANT', "HENRY'S", 'HERMOSA', 'HEROS', 'M  HENRIETTA', 'M. HENRY']

